In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-04 10:05:10.834581: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 10:05:10.866050: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 10:05:10.866753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 10:05:12.216517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
955


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2838                 251.444  -     23   +     24       False
background_yield_0.2838             903.437  -     35   +     35       False
frac_Mbc_bkg1_0.2838               0.578282  -   0.05   +  0.049       False
frac_deltaE_bkg1_0.2838            0.192388  -   0.14   +   0.14       False
True
Iteration 1 - Signal Yield: 251.44400928162864 ± 23.90623924623123
Running iteration 2/300...
200
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9682                 225.962  -     23   +     24       False
background_yield_0.9682              929.03  -     35   +     36       False
frac_Mbc_bkg1_0.9682                0.61686  -  0.049   +  0.047       False
frac_deltaE_bkg1_0.9682            0.191804  -

## 150

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2539                  197.73  -     22   +     23       False
background_yield_0.2539             907.266  -     34   +     35       False
frac_Mbc_bkg1_0.2539               0.583484  -   0.05   +  0.049       False
frac_deltaE_bkg1_0.2539            0.197333  -   0.14   +   0.14       False
True
Iteration 1 - Signal Yield: 197.72953475488526 ± 22.394351704978405
Running iteration 2/300...
150
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6929                 174.792  -     22   +     22       False
background_yield_0.6929             930.357  -     35   +     36       False
frac_Mbc_bkg1_0.6929                0.61896  -  0.048   +  0.048       False
frac_delta

## 100

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2725                 142.794  -     20   +     21       False
background_yield_0.2725             912.199  -     34   +     35       False
frac_Mbc_bkg1_0.2725                0.58924  -  0.049   +  0.048       False
frac_deltaE_bkg1_0.2725            0.192869  -   0.14   +   0.14       False
True
Iteration 1 - Signal Yield: 142.7941306348446 ± 20.64451850185916
Running iteration 2/300...
100
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5513                  125.42  -     20   +     21       False
background_yield_0.5513             929.546  -     35   +     36       False
frac_Mbc_bkg1_0.5513               0.618844  -  0.048   +  0.047       False
frac_deltaE_

## 75

In [5]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4463                 117.838  -     19   +     20       False
background_yield_0.4463             912.148  -     34   +     35       False
frac_Mbc_bkg1_0.4463               0.589773  -  0.049   +  0.048       False
frac_deltaE_bkg1_0.4463            0.191382  -   0.14   +   0.13       False
True
Iteration 1 - Signal Yield: 117.83808236393644 ± 19.806008383361338
Running iteration 2/300...
75
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9357                 104.438  -     19   +     20       False
background_yield_0.9357             925.559  -     34   +     35       False
frac_Mbc_bkg1_0.9357               0.612286  -  0.048   +  0.047       False
frac_deltaE_

## 50

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
955
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.591                 93.6098  -     19   +     19       False
background_yield_0.591             911.443  -     34   +     35       False
frac_Mbc_bkg1_0.591               0.590021  -  0.048   +  0.048       False
frac_deltaE_bkg1_0.591            0.203783  -   0.14   +   0.13       False
True
Iteration 1 - Signal Yield: 93.6098196946293 ± 18.91518102962629
Running iteration 2/300...
50
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4509                 73.5902  -     19   +     19       False
background_yield_0.4509             931.409  -     34   +     35       False
frac_Mbc_bkg1_0.4509               0.619566  -  0.047   +  0.046       False
frac_deltaE_bkg1_0.45

## 25

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2395                 66.8955  -     17   +     18       False
background_yield_0.2395             912.286  -     33   +     35       False
frac_Mbc_bkg1_0.2395               0.593363  -  0.048   +  0.047       False
frac_deltaE_bkg1_0.2395            0.196945  -   0.14   +   0.13       False
True
Iteration 1 - Signal Yield: 66.89548678293329 ± 17.87380749902668
Running iteration 2/300...
25
955
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.519                 51.5657  -     17   +     18       False
background_yield_0.519             928.029  -     34   +     35       False
frac_Mbc_bkg1_0.519               0.615574  -  0.048   +  0.046       False
frac_deltaE_bkg1_0.

## 10

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3374                 48.1205  -     17   +     18       False
background_yield_0.3374             916.782  -     34   +     35       False
frac_Mbc_bkg1_0.3374               0.599135  -  0.049   +  0.046       False
frac_deltaE_bkg1_0.3374            0.220961  -   0.14   +   0.13       False
True
Iteration 1 - Signal Yield: 48.120485853045714 ± 17.27680066814875
Running iteration 2/300...
10
955
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.715                 38.4356  -     17   +     17       False
background_yield_0.715             926.596  -     34   +     35       False
frac_Mbc_bkg1_0.715               0.611733  -  0.047   +  0.046       False
frac_deltaE_bkg1_0

## 5

In [9]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1825                 40.0891  -     17   +     17       False
background_yield_0.1825             919.824  -     34   +     35       False
frac_Mbc_bkg1_0.1825               0.602471  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.1825            0.227311  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 40.08905663175715 ± 17.018765103259256
Running iteration 2/300...
5
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4679                 35.1726  -     16   +     17       False
background_yield_0.4679               924.8  -     34   +     35       False
frac_Mbc_bkg1_0.4679               0.609121  -  0.048   +  0.046       False
frac_deltaE_bkg

## 2

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
955


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4856                 34.4706  -     17   +     17       False
background_yield_0.4856             922.431  -     34   +     35       False
frac_Mbc_bkg1_0.4856               0.605868  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.4856            0.230561  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 34.47060648609455 ± 16.854376985462018
Running iteration 2/300...
2
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0317                 33.3193  -     17   +     17       False
background_yield_0.0317             923.572  -     34   +     35       False
frac_Mbc_bkg1_0.0317               0.607176  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.0317            0.238849  -  

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppi0,file_bkg_ele_Bppi0, 'ele', '521', 'pi0', 'ee', 3, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppi0_q2reg3_decay_ee_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppi0_q2reg3_decay_ee_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
955


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7602                 34.3954  -     17   +     17       False
background_yield_0.7602             920.538  -     34   +     35       False
frac_Mbc_bkg1_0.7602               0.602995  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.7602            0.230364  -   0.13   +   0.13       False
True
Iteration 1 - Signal Yield: 34.395387426277416 ± 16.78631639155098
Running iteration 2/300...
0
955
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3349                 34.3954  -     17   +     17       False
background_yield_0.3349             920.538  -     34   +     35       False
frac_Mbc_bkg1_0.3349               0.602995  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.3349            0.230364  -  